In [1]:
# %load ../../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)
 
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

Numericas:
-  'Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       

Categoricas:
 - 'Month': ()
 - 'Weekend': 1(Fasle), 2(True)
 - 'VisitorType': 1(Ha vuelto), 2(Nuevo), 3(otro)

Clases 'Revenue'

In [52]:
datos = pd.read_csv('online_shoppers_intention.csv')

datos_ok = datos.copy()
print(datos_ok.head())
categorico = ['Month','Weekend','VisitorType','Revenue']

for i,val in enumerate(datos_ok.columns):
    if val == 'Month':
        mesesDic = dict(zip(['Feb', 'Mar', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], [2,3,5,6,7,8,9,10,11,12]))
        datos_ok[val].replace(mesesDic, inplace=True)
    elif val in categorico:
        labels = datos_ok[val].astype('category').cat.categories.tolist()
        dict_make = {val : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
        datos_ok[val].replace(dict_make[val], inplace=True)

X_all = np.array(datos_ok.drop(columns = 'Revenue').copy())
y_all = np.array(pd.factorize(datos_ok['Revenue'])[0])

print(X_all[:5,:])

Administrative  Administrative_Duration  Informational  \
0             0.0                      0.0            0.0   
1             0.0                      0.0            0.0   
2             0.0                     -1.0            0.0   
3             0.0                      0.0            0.0   
4             0.0                      0.0            0.0   

   Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                     0.0             1.0                 0.000000   
1                     0.0             2.0                64.000000   
2                    -1.0             1.0                -1.000000   
3                     0.0             2.0                 2.666667   
4                     0.0            10.0               627.500000   

   BounceRates  ExitRates  PageValues  SpecialDay Month  OperatingSystems  \
0         0.20       0.20         0.0         0.0   Feb                 1   
1         0.00       0.10         0.0         0.0   Feb      

NameError: name 'mesesDic' is not defined

Elimnacion de valores Perididos

In [37]:
n = np.isnan(X_all)
n = np.sum(n,axis=1).reshape(-1,1).astype('bool')
n = n == False
X = X_all[n.flatten()]
y = y_all[n.flatten()]

## One Hot Ecnode busqueda

In [38]:
#hanwei
# Se importan las 3 librerías necesarias para resolver el ejercicio
from sklearn import neighbors, metrics, model_selection
import sklearn.tree as tree
print(X_all.shape)
n = np.isnan(X_all)

# Se fija la semilla de numpy para que la generación aleatoria siempre nos de los mismos números
np.random.seed(12)

# Lllamada a la función train_test_split y guardado del resultado
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

# Realizamos el proceso para KNN por lo que hay que llamar al constructor de dicho clasificador
clf = tree.DecisionTreeClassifier()

# Llamada a la función que realiza el aprendizaje del clasificador
clf.fit(X_train,y_train)

# Llamada a la función que realiza la predicción de los datos de entrenamiento
predTrain = clf.predict(X_train)

# Llamada a la función que calcula el porcentaje de acierto para los datos de entrenamiento
accTrain = round(metrics.accuracy_score(y_train,predTrain)*100,2)
print('El rendimiento en entrenamiento con todas las variables es el {}%'.format(accTrain))

# Llamada a la función que realiza la predicción de los datos de test
predTest = clf.predict(X_test)

# Llamada a la función que calcula el porcentaje de acierto para los datos de test
accTest = round(metrics.accuracy_score(y_test,predTest)*100,2)
print('El rendimiento en test con todas las variables es el {}%'.format(accTest))

(12330, 17)
El rendimiento en entrenamiento con todas las variables es el 100.0%
El rendimiento en test con todas las variables es el 85.9%


In [23]:
num = np.sum(datos['Revenue'] == 1)
num2 = np.sum(datos['Revenue'] == 0)
print(num,num2)
print(y_all)

x_pos = X_all[y_all == 1]
x_pos = X_all[y_all == 0]

1908 10422
[0 0 0 ... 0 0 0]


In [50]:
val = 'Month'
labels = datos_ok[val].astype('category').cat.categories.tolist()
dict_make = {val : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
pd.factorize(datos[val])
# pd.factorize(sonar[' Type'])[0]
